# **Ambulance Placement System in a Large City**

## **Introduction**
In a large city, efficient placement of ambulances is critical for quick emergency response. This system divides the city into smaller localities and calculates scores based on various parameters such as population, age distribution, socioeconomic status, road size, and proximity to hospitals. These scores help determine the optimal locations and types of ambulances to be deployed.

## **Idea**
First the ambulances are placed on the map based on the data / history and then the route optimization is applied. The system optimizes ambulance placement and emergency response by leveraging real-time data, predictive analytics, and optimization algorithms. It collects and processes various data sources, including GPS data, Google Maps reviews, and historical emergency response data, to forecast emergency hotspots and optimize ambulance placement. The system then tracks ambulances in real-time, optimizes routes, and distributes patients efficiently across hospitals. Continuous monitoring and evaluation refine the system, ensuring improved response times, accurate predictions, and effective resource allocation. By integrating these components, the system enhances emergency response and saves lives.

## **1. Data Preparation**
To start, we need datasets that provide information about the city’s population, age distribution, socioeconomic status, road network, and hospitals. These datasets will be used to divide the city into localities and calculate the necessary parameters.

### **Loading the Data**

In [ ]:
import pandas as pd

In [ ]:
# Load the datasets (replace with actual file paths)
#population_data = pd.read_csv('population_data.csv')
age_data = pd.read_csv('age_data.csv')
socioeconomic_data = pd.read_csv('socioeconomic_data.csv')
road_data = pd.read_csv('road_data.csv')
hospital_data = pd.read_csv('hospital_data.csv')
city_map = gpd.read_file('city_map.geojson')  # City boundary GeoJSON file

In [ ]:
!pip install gmplot geopy

In [1]:
from gmplot import gmplot
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import numpy as np


In [ ]:
pip install folium

In [15]:
import folium
import pandas as pd
import re

# Initialize the map centered at Bangalore
bangalore_center = [12.9716, 77.5946]
bangalore_map = folium.Map(location=bangalore_center, zoom_start=11)

# Define the zones with approximate rectangular boundaries
zones = {
    "Central Bangalore": {
        "coordinates": [
            [12.96, 77.56],
            [12.96, 77.62],
            [13.00, 77.62],
            [13.00, 77.56],
        ],
        "color": "blue",
        "fill_opacity": 0.4
    },
    "North Bangalore": {
        "coordinates": [
            [13.00, 77.56],
            [13.00, 77.62],
            [13.05, 77.62],
            [13.05, 77.56],
        ],
        "color": "green",
        "fill_opacity": 0.4
    },
    "South Bangalore": {
        "coordinates": [
            [12.90, 77.56],
            [12.90, 77.62],
            [12.96, 77.62],
            [12.96, 77.56],
        ],
        "color": "red",
        "fill_opacity": 0.4
    },
    "East Bangalore": {
        "coordinates": [
            [12.96, 77.62],
            [12.96, 77.78],
            [13.05, 77.78],
            [13.05, 77.62],
        ],
        "color": "purple",
        "fill_opacity": 0.4
    },
    "West Bangalore": {
        "coordinates": [
            [12.96, 77.40],
            [12.96, 77.56],
            [13.05, 77.56],
            [13.05, 77.40],
        ],
        "color": "orange",
        "fill_opacity": 0.4
    },
    "Northeast Bangalore": {
        "coordinates": [
            [13.00, 77.62],
            [13.00, 77.78],
            [13.05, 77.78],
            [13.05, 77.62],
        ],
        "color": "darkgreen",
        "fill_opacity": 0.4
    },
    "Southeast Bangalore": {
        "coordinates": [
            [12.90, 77.62],
            [12.90, 77.78],
            [12.96, 77.78],
            [12.96, 77.62],
        ],
        "color": "cadetblue",
        "fill_opacity": 0.4
    },
    "Southwest Bangalore": {
        "coordinates": [
            [12.90, 77.40],
            [12.90, 77.56],
            [12.96, 77.56],
            [12.96, 77.40],
        ],
        "color": "darkred",
        "fill_opacity": 0.4
    },
    "Outer Bangalore": {
        "coordinates": [
            [12.80, 77.30],
            [12.80, 77.90],
            [13.15, 77.90],
            [13.15, 77.30],
        ],
        "color": "lightgray",
        "fill_opacity": 0.1,
        "fill": False  # Set to False to avoid overlapping fills
    }
}

# Function to add zones to the map
def add_zones(map_obj, zones_dict):
    for zone_name, zone_info in zones_dict.items():
        coords = zone_info["coordinates"]
        color = zone_info["color"]
        fill_opacity = zone_info.get("fill_opacity", 0.3)
        fill = zone_info.get("fill", True)
        
        folium.Polygon(
            locations=coords,
            color=color,
            weight=2,
            fill=fill,
            fill_color=color if fill else None,
            fill_opacity=fill_opacity,
            tooltip=zone_name,
        ).add_to(map_obj)

# Add all zones to the map
add_zones(bangalore_map, zones)

# Function to convert coordinate strings to decimal format
def convert_to_decimal(coordinate):
    match = re.match(r"(\d+\.\d+)° N, (\d+\.\d+)° E", coordinate)
    if match:
        return float(match.group(1)), float(match.group(2))
    return None, None

# Load the accident-prone areas data from the CSV file
file_path = 'accident prone areas_bangalore.csv'
data = pd.read_csv(file_path)

# Extract latitude and longitude for each area
data['Latitude'], data['Longitude'] = zip(*data['Approximate Coordinates'].map(convert_to_decimal))

# Remove rows with NaN values
data.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Plot each accident-prone area on the map
for lat, lon in zip(data['Latitude'], data['Longitude']):
    folium.Marker([lat, lon], icon=folium.Icon(color='red')).add_to(bangalore_map)

# Add a layer control to toggle zones if needed
folium.LayerControl().add_to(bangalore_map)

# Save the map to an HTML file
bangalore_map.save("bangalore_zones_accident_prone_areas_map_1.html")

print("Map has been saved to 'bangalore_zones_accident_prone_areas_map.html'. Open this file in a web browser to view the map.")

Map has been saved to 'bangalore_zones_accident_prone_areas_map.html'. Open this file in a web browser to view the map.


In [16]:
# Load the CSV file - Bangalore City EDA for Socioeconmic data
df = pd.read_csv('/content/Bengaluru_House_Data.csv')

# Ensure columns are correctly named
df.columns = ['location', 'size', 'total_sqft', 'price']

# Convert 'total_sqft' and 'price' to numeric values
df['total_sqft'] = pd.to_numeric(df['total_sqft'], errors='coerce')
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Calculate price per square foot
df['price_per_sqft'] = df['price'] / df['total_sqft']

# Drop rows with invalid 'price_per_sqft'
df = df.dropna(subset=['price_per_sqft'])

# Group by location and calculate the average price per square foot
location_price_per_sqft = df.groupby('location')['price_per_sqft'].mean().reset_index()

# Sort localities by price per square foot
sorted_locations = location_price_per_sqft.sort_values(by='price_per_sqft', ascending=False)

# Display the sorted locations
print(sorted_locations)

# Optionally, save the result to a new CSV file
sorted_locations.to_csv('sorted_localities_by_price_per_sqft.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Bengaluru_House_Data.csv'

### Web scraping real-time stats for key areas
```python
# Here we are getting the road/traffic conditions using Twitter/Web
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

top_5_locations = sorted_locations.head(5)['location'].tolist()

# Step 3: Use Selenium to search for road quality for the top 5 locations
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

road_quality_info = {}

for location in top_5_locations:
    search_query = f"{location} road quality"
    driver.get(f"https://www.google.com/search?q={search_query}")
    try:
        snippet = driver.find_element(By.XPATH, '//div[@class="BNeawe s3v9rd AP7Wnd"]').text
        road_quality_info[location] = snippet
    except:
        road_quality_info[location] = "No information found"

driver.quit()

# Print road quality information
for location, info in road_quality_info.items():
    print(f"Location: {location}\nRoad Quality: {info}\n")

# Step 4: Rank Locations Based on Road Quality
ranked_locations = sorted(road_quality_info.items(), key=lambda item: item[1], reverse=True)
print("Ranked Locations Based on Road Quality:")
for rank, (location, info) in enumerate(ranked_locations, start=1):
    print(f"{rank}. {location}: {info}")

```

# 2. Dividing the City into Localities
We divide the city into smaller localities. This can be done using a grid, existing administrative boundaries, or other methods. For simplicity, we use the centroids of city regions.

In [ ]:
# Divide the city into localities based on the map geometry
localities = city_map['geometry'].apply(lambda x: x.centroid)  # Simple centroid-based division

# 3. Calculating Parameters for Each Locality
Next, we calculate key parameters for each locality: population, average age, socioeconomic status, road size, and hospital type.

In [ ]:
def calculate_locality_data(locality_geom, population_data, age_data, socioeconomic_data, road_data, hospital_data):
    population = population_data[population_data['geometry'].within(locality_geom)]['population'].sum()
    age_group = age_data[age_data['geometry'].within(locality_geom)]['age'].mean()
    socioeconomic_status = socioeconomic_data[socioeconomic_data['geometry'].within(locality_geom)]['socioeconomic'].mean()
    road_size = road_data[road_data['geometry'].within(locality_geom)]['road_width'].mean()
    nearest_hospital = hospital_data[hospital_data['geometry'].nearest(locality_geom)]['hospital_type'].values[0]

    return {
        'Population': population,
        'Age_Group': age_group,
        'Socioeconomic_Status': socioeconomic_status,
        'Road_Size': road_size,
        'Hospital_Type': nearest_hospital
    }

# Apply the function to each locality
locality_scores = []
for locality in localities:
    locality_data = calculate_locality_data(locality, population_data, age_data, socioeconomic_data, road_data, hospital_data)
    locality_scores.append(locality_data)

locality_scores_df = pd.DataFrame(locality_scores)


# 4. Scoring Localities
With the calculated parameters, we now score each locality to determine the need and type of ambulance

### Scoring Function


In [ ]:
def score_locality(row):
    score = (
        row['Population'] * 0.4 +
        row['Age_Group'] * 0.3 +
        row['Socioeconomic_Status'] * 0.2 +
        row['Road_Size'] * 0.1
    )
    return score

locality_scores_df['Score'] = locality_scores_df.apply(score_locality, axis=1)

# This function assigns a score to each locality based on the weighted parameters. The weights can be adjusted based on the specific needs of your city.


# 5. Determining Ambulance Type
The type of ambulance to deploy is determined based on the locality's score, road size, and hospital type.

### Ambulance Type Determination

In [ ]:
def determine_ambulance_type(row):
    if row['Hospital_Type'] == 'Super Speciality':
        return 'Large, ALS' if row['Road_Size'] >= 6 else 'Small, ALS'
    elif row['Hospital_Type'] == 'Cardiac':
        return 'Cardiac Ambulance' if row['Road_Size'] >= 6 else 'Small, Cardiac Ambulance'
    elif row['Hospital_Type'] == 'Eye':
        return 'Basic Ambulance' if row['Road_Size'] >= 6 else 'Small, Basic Ambulance'
    else:
        return 'Standard Ambulance' if row['Road_Size'] >= 6 else 'Small Ambulance'

locality_scores_df['Ambulance_Type'] = locality_scores_df.apply(determine_ambulance_type, axis=1)


# 6. Placing Ambulances
Finally, we allocate the ambulances to the localities based on their scores and the determined ambulance type.

### Ambulance Placement

In [ ]:
def place_ambulances(locality_scores_df):
    placements = []
    for index, row in locality_scores_df.iterrows():
        placements.append((row['Locality'], row['Ambulance_Type'], row['Score']))
    return placements

placements = place_ambulances(locality_scores_df)
placements


This step finalizes the placement of ambulances. It provides a list of localities with the recommended type of ambulance and the calculated score for each.